In [ ]:
import pandas as pd
import keras
import tensorflow as tf
from keras import layers
from tensorflow.keras.layers import Dense
import numpy as np
import random
import matplotlib.pyplot as plt

In [ ]:
def convert_str_to_list(strng) :
  strng = strng.replace('[', '').replace(']', '')
  separation = strng.split(", ")
  return separation

convert_str_to_list("[1234, 134, 345]")

['1234', '134', '345']

In [ ]:
df = pd.read_csv("/content/pa_listEspeces_donneesAbiotiques.csv", sep=",", na_values=['NA', 'NaN'])

df["speciesId"] = df["speciesId"].apply(convert_str_to_list)

# Appliquer la conversion de chaîne à liste sur la colonne "speciesId"
df["speciesId"] = df["speciesId"].apply(lambda x: [str(i) for i in x])

# Obtenez toutes les espèces uniques
all_species = set(species for sublist in df['speciesId'] for species in sublist)

# Créer une colonne par espèce avec des indicateurs 0 ou 1
for species in all_species:
    df[species] = df['speciesId'].apply(lambda x: 1 if species in x else 0)

# Supprimer la colonne d'origine 'speciesId'
df = df.drop(columns=['speciesId'])
df = df.dropna().reset_index()

# Afficher le DataFrame résultant
print(df)



<ipython-input-4-a0ca899ec1de>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[species] = df['speciesId'].apply(lambda x: 1 if species in x else 0)
<ipython-input-4-a0ca899ec1de>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[species] = df['speciesId'].apply(lambda x: 1 if species in x else 0)
<ipython-input-4-a0ca899ec1de>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.

      index  Unnamed: 0  patchID  dayOfYear      lon       lat    x_EPSG3035  \
0         0           0    55880        126  4.74408  44.20370  3.900369e+06   
1         1           1    55897        126  4.74436  44.20410  3.900395e+06   
2         2           2    55942        126  4.74519  44.20360  3.900457e+06   
3         3           3    56533        126  4.75466  44.21930  3.901329e+06   
4         4           4    56759        126  4.75755  44.21360  3.901518e+06   
...     ...         ...      ...        ...      ...       ...           ...   
5211   5943        5943  3018845        138  9.28662  41.87388  4.261560e+06   
5212   5944        5944  3018847        351  9.29834  41.56691  4.262244e+06   
5213   5945        5945  3018848        346  9.31529  41.70986  4.263796e+06   
5214   5946        5946  3018853        346  9.35871  41.67093  4.267389e+06   
5215   5947        5947  3018859        165  9.39303  41.95686  4.270495e+06   

        y_EPSG3035  year               

In [ ]:
X = df.iloc[:,0:50]
y = df.iloc[:, 50:]
X


,index,Unnamed: 0,patchID,dayOfYear,lon,lat,x_EPSG3035,y_EPSG3035,year,dataset,...,Built1994,Lights1994,NavWater1994,Built2009,Lights2009,NavWater2009,Popdensity1990,Popdensity2010,Railways,Roads
0,0,0,55880,126,4.74408,44.20370,3.900369e+06,2.358486e+06,2020,CBNMed,...,0.0,7.0,0.357577,10.0,8.0,0.357577,6.0,6.0,0.0,1.629548
1,1,1,55897,126,4.74436,44.20410,3.900395e+06,2.358529e+06,2020,CBNMed,...,0.0,7.0,0.357577,10.0,8.0,0.357577,6.0,6.0,0.0,1.629548
2,2,2,55942,126,4.74519,44.20360,3.900457e+06,2.358469e+06,2020,CBNMed,...,0.0,7.0,0.357577,10.0,8.0,0.357577,6.0,6.0,0.0,1.629548
3,3,3,56533,126,4.75466,44.21930,3.901329e+06,2.360153e+06,2020,CBNMed,...,0.0,4.0,0.144801,0.0,5.0,0.144801,6.0,6.0,0.0,0.852498
4,4,4,56759,126,4.75755,44.21360,3.901518e+06,2.359506e+06,2020,CBNMed,...,0.0,4.0,0.144801,0.0,5.0,0.144801,6.0,6.0,0.0,0.852498
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5211,5943,5943,3018845,138,9.28662,41.87388,4.261560e+06,2.085921e+06,2018,Inventaire forestier IGN,...,0.0,0.0,0.002858,0.0,0.0,0.002858,2.0,2.0,0.0,0.757507
5212,5944,5944,3018847,351,9.29834,41.56691,4.262244e+06,2.051949e+06,2018,Inventaire forestier IGN,...,0.0,3.0,1.281452,0.0,6.0,1.281452,5.0,6.0,0.0,1.629548
5213,5945,5945,3018848,346,9.31529,41.70986,4.263796e+06,2.067751e+06,2017,Inventaire forestier IGN,...,0.0,1.0,0.042134,0.0,1.0,0.042134,4.0,4.0,0.0,4.000000
5214,5946,5946,3018853,346,9.35871,41.67093,4.267389e+06,2.063412e+06,2018,Inventaire forestier IGN,...,0.0,3.0,1.788531,0.0,4.0,1.788531,3.0,4.0,0.0,0.526984


In [ ]:
X = X.drop(columns="dataset")
X = X.rename(columns={"Unnamed: 0":'Id'})
X = X.drop(columns="x_EPSG3035")
X = X.drop(columns="y_EPSG3035")
X = X.drop(columns="patchID")
X = X.drop(columns="geoUncertaintyInM")
X = X.drop(columns='Id')
X = X.drop(columns='index')
X

,dayOfYear,lon,lat,year,bio1,bio2,bio3,bio4,bio5,bio6,...,Built1994,Lights1994,NavWater1994,Built2009,Lights2009,NavWater2009,Popdensity1990,Popdensity2010,Railways,Roads
0,126,4.74408,44.20370,2020,2878,95,3.33,6635,3032,2747,...,0.0,7.0,0.357577,10.0,8.0,0.357577,6.0,6.0,0.0,1.629548
1,126,4.74436,44.20410,2020,2878,95,3.33,6635,3032,2747,...,0.0,7.0,0.357577,10.0,8.0,0.357577,6.0,6.0,0.0,1.629548
2,126,4.74519,44.20360,2020,2878,95,3.33,6635,3032,2747,...,0.0,7.0,0.357577,10.0,8.0,0.357577,6.0,6.0,0.0,1.629548
3,126,4.75466,44.21930,2020,2876,95,3.33,6614,3031,2746,...,0.0,4.0,0.144801,0.0,5.0,0.144801,6.0,6.0,0.0,0.852498
4,126,4.75755,44.21360,2020,2877,95,3.33,6635,3031,2746,...,0.0,4.0,0.144801,0.0,5.0,0.144801,6.0,6.0,0.0,0.852498
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5211,138,9.28662,41.87388,2018,2839,86,3.31,5862,2981,2723,...,0.0,0.0,0.002858,0.0,0.0,0.002858,2.0,2.0,0.0,0.757507
5212,351,9.29834,41.56691,2018,2898,40,2.14,5081,3000,2815,...,0.0,3.0,1.281452,0.0,6.0,1.281452,5.0,6.0,0.0,1.629548
5213,346,9.31529,41.70986,2017,2894,76,3.09,5864,3030,2785,...,0.0,1.0,0.042134,0.0,1.0,0.042134,4.0,4.0,0.0,4.000000
5214,346,9.35871,41.67093,2018,2897,62,2.80,5587,3021,2798,...,0.0,3.0,1.788531,0.0,4.0,1.788531,3.0,4.0,0.0,0.526984


In [ ]:
X_base = X[["lon", "lat", 'bio1', 'bio2', 'bio3', 'bio4', 'bio5', 'bio6', 'bio7', 'bio8', 'bio9', 'bio10', 'bio11', 'bio12', 'bio13', 'bio14', 'bio15', 'bio16', 'bio17', 'bio18', 'bio19'
              ,'bdod', 'cec', 'cfvo', 'clay', 'nitrogen', 'phh2o', 'sand', 'silt', 'soc', 'landCov', 'Built1994', 'Lights1994', 'NavWater1994', 'Built2009','Lights2009', 'NavWater2009',
             'Popdensity1990', 'Popdensity2010','Railways', 'Roads']]


matrix = []
for i in range(len(X_base)):
  new_liste = []
  #for col in X_base.columns :
  for col in  X_base.columns :
    new_liste.append(X_base[col][i])
  matrix.append(new_liste)


#matrix_y = []
#for i in range(len(y)):
#  new_liste = []
#  for col in y.columns :
#    new_liste.append(y[col][i])
#  matrix_y.append(new_liste)

#matrix_y


#yt = y.iloc[:, 1].to_list()
#print(sum(yt))
#Xt = np.asarray(np.array(matrix))
#Xt

In [ ]:
#yt = np.array(y.iloc[:,0]).astype(np.float32)
yt = np.array(y).astype(np.float32)
print(len(yt[0]))
print(len(X_base.columns))

2174
41


In [ ]:
Xt = np.array(matrix)
Xt = Xt.astype(np.float32)
len(Xt)

5216

In [ ]:
model = keras.Sequential()
model.add(keras.Input(shape=(41,)))
model.add(layers.Dense(32, activation="relu"))
model.add(layers.Dense(64, activation="sigmoid"))
model.add(layers.Dense(32, activation="PReLU"))
#model.add(layers.Dense(2048, activation="tanh"))
model.add(layers.Dense(2174, activation="sigmoid", name='output'))
#model.add(layers.Dense(4096, activation='tanh'))
#model.add(layers.Dense(8192, activation='PReLU'))


#y_v = model(Xt)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                1344      
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 dense_2 (Dense)             (None, 32)                2112      
                                                                 
 output (Dense)              (None, 2174)              71742     
                                                                 
Total params: 77310 (301.99 KB)
Trainable params: 77310 (301.99 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="binary_crossentropy",
    metrics=[
       tf.keras.metrics.F1Score()
       ],
    run_eagerly=True
  )

In [ ]:
print("Fit model on training data")
history = model.fit(
    Xt,
    yt,
    batch_size=10,
    epochs=2,
)

Fit model on training data
Epoch 1/2
522/522 [==============================] - 25s 47ms/step - loss: 0.0313 - f1_score: 3.6510e-04
Epoch 2/2
522/522 [==============================] - 25s 47ms/step - loss: 0.0312 - f1_score: 3.9356e-04


In [ ]:
df_test = pd.read_csv("/content/enviroTab_pa_test.csv", sep=';')
df_test = df_test[["lon", "lat", 'bio1', 'bio2', 'bio3', 'bio4', 'bio5', 'bio6', 'bio7', 'bio8', 'bio9', 'bio10', 'bio11', 'bio12', 'bio13', 'bio14', 'bio15', 'bio16', 'bio17', 'bio18', 'bio19'
              ,'bdod', 'cec', 'cfvo', 'clay', 'nitrogen', 'phh2o', 'sand', 'silt', 'soc', 'landCov', 'Built1994', 'Lights1994', 'NavWater1994', 'Built2009','Lights2009', 'NavWater2009',
             'Popdensity1990', 'Popdensity2010','Railways', 'Roads']]

X_test = []
for i in range(len(df_test)):
  new_liste = []
  for col in df_test.columns :
    new_liste.append(df_test[col][i])
  X_test.append(new_liste)

In [ ]:
X_t = np.array(X_test)
X_t = X_t.astype(np.float32)

In [ ]:
pred = model.predict(X_t)

701/701 [==============================] - 5s 8ms/step


In [ ]:
ids_esp = y.columns.to_list()
nb_esp = 20
tops_k = []
for i in range(len(pred)) :
  dictio = {}
  top_k = []
  for j in range(len(pred[i])):
    dictio[pred[i][j]] = ids_esp[j]
  while len(top_k) < nb_esp :


In [ ]:
# Option 2 :

ids_esp = y.columns.to_list()
tops_25 = []
for i in range(len(pred)) :
  print(i)
  top_25 = []
  indice_ajt = []
  while len(top_25)<100 :
    max_value = 0
    indice_max = 0
    for j in range(len(pred[i])):
      if pred[i][j] > max_value and j not in indice_ajt :
        max_value = pred[i][j]
        indice_max = j
    indice_ajt.append(indice_max)
    top_25.append(ids_esp[indice_max])
  tops_25.append(top_25)

tops_25

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
17404
17405
17406
17407
17408
17409
17410
17411
17412
17413
17414
17415
17416
17417
17418
17419
17420
17421
17422
17423
17424
17425
17426
17427
17428
17429
17430
17431
17432
17433
17434
17435
17436
17437
17438
17439
17440
17441
17442
17443
17444
17445
17446
17447
17448
17449
17450
17451
17452
17453
17454
17455
17456
17457
17458
17459
17460
17461
17462
17463
17464
17465
17466
17467
17468
17469
17470
17471
17472
17473
17474
17475
17476
17477
17478
17479
17480
17481
17482
17483
17484
17485
17486
17487
17488
17489
17490
17491
17492
17493
17494
17495
17496
17497
17498
17499
17500
17501
17502
17503
17504
17505
17506
17507
17508
17509
17510
17511
17512
17513
17514
17515
17516
17517
17518
17519
17520
17521
17522
17523
17524
17525
17526
17527
17528
17529
17530
17531
17532
17533
17534
17535
17536
17537
17538
17539
17540
17541
17542
17543
17544
17545
17546
17547
17548
17549
17550
17551
17552
17553
17554
17555
17556
1755

[['4284',
  '8935',
  '5959',
  '6837',
  '5314',
  '386',
  '9906',
  '2509',
  '5763',
  '9330',
  '1005',
  '9347',
  '9329',
  '515',
  '7648',
  '801',
  '795',
  '3069',
  '9474',
  '1849',
  '5263',
  '8987',
  '4906',
  '8079',
  '8741',
  '5989',
  '1662',
  '24',
  '7570',
  '2855',
  '8999',
  '9118',
  '6361',
  '3833',
  '4195',
  '7256',
  '1333',
  '4799',
  '7934',
  '2802',
  '99',
  '6574',
  '9075',
  '4634',
  '4218',
  '7426',
  '7787',
  '268',
  '1283',
  '2854',
  '1384',
  '7822',
  '4064',
  '1889',
  '8419',
  '254',
  '307',
  '849',
  '3097',
  '6107',
  '1913',
  '2622',
  '2231',
  '793',
  '7367',
  '3814',
  '2098',
  '5707',
  '4051',
  '3408',
  '8390',
  '8226',
  '4059',
  '4967',
  '8176',
  '8124',
  '4182',
  '8395',
  '890',
  '7415',
  '8914',
  '904',
  '5154',
  '265',
  '8058',
  '7747',
  '9993',
  '4408',
  '729',
  '1170',
  '9011',
  '5644',
  '303',
  '9823',
  '6705',
  '6731',
  '5404',
  '6226',
  '4723',
  '6473'],
 ['4284',
  '8935

In [ ]:
top_25_save = top_25.copy()

In [ ]:
tops_20 = []
for i in range(len(top_25_save)):
  new_list = top_25_save[i][0:20]
  tops_20.append(new_list)

In [ ]:
def liste_to_str(arr):
  chaine = ""
  for i in range(len(arr)):
    chaine+= arr[i]+" "
  return chaine

In [ ]:
submission = pd.read_csv("/content/enviroTab_pa_test.csv", sep=';')




colonne_reponse = []
for i in range(len(tops_25)):
  colonne_reponse.append(liste_to_str(tops_25[i]))

submission["Predicted"] = colonne_reponse

submission = submission[["Id", "Predicted"]]
submission.to_csv("/content/deep_pa.csv", sep=',', index=False)